In [1]:
%pylab inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = pd.read_csv("../../Dataset/Bag Of Words/labeledTrainData.tsv",delimiter='\t',header=0,quoting=3)
print train.shape

(25000, 3)


In [3]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [5]:
def clean_review(review):
    review = BeautifulSoup(review).get_text()
    letters_only = re.sub("[^A-Za-z]"," ",review)
    lower_case = letters_only.lower()
    words = lower_case.split()
    return " ".join([word for word in words if word not in set(stopwords.words("english"))])

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import os.path
review = []
print "Cleaning and parsing the training set movie reviews...."
if os.path.exists('trainLabeledData.csv'):
    print 'File already exists'
else:
    for i in range(len(train["review"])):
        try:
            review.append(clean_review(train["review"][i]))
        except:
            print 'Error occured for {} review...'.format(i)
        if i % 1000 == 0:
            print '{} reviews parsed...'.format(i)

Cleaning and parsing the training set movie reviews....
0 reviews parsed...
1000 reviews parsed...
2000 reviews parsed...
3000 reviews parsed...
4000 reviews parsed...
5000 reviews parsed...
6000 reviews parsed...
7000 reviews parsed...
8000 reviews parsed...
9000 reviews parsed...
10000 reviews parsed...
11000 reviews parsed...
12000 reviews parsed...
13000 reviews parsed...
14000 reviews parsed...
15000 reviews parsed...
16000 reviews parsed...
17000 reviews parsed...
18000 reviews parsed...
19000 reviews parsed...
20000 reviews parsed...
21000 reviews parsed...
22000 reviews parsed...
23000 reviews parsed...
24000 reviews parsed...


In [12]:
train["review"] = review

In [13]:
if os.path.exists('trainLabeledData.csv'):
    pass
else:
    train.to_csv("trainLabeledData.csv",header=train.columns,sep='\t')

In [14]:
print "Creating the Bag Of Words Representation....."
from sklearn.feature_extraction.text import CountVectorizer

Creating the Bag Of Words Representation.....


In [15]:
vectorizer = CountVectorizer(analyzer="word",tokenizer=None,preprocessor=None,stop_words=None,max_features=5000)
train_data_features = vectorizer.fit_transform(train["review"].tolist())

In [16]:
train_data_features = train_data_features.toarray()

In [17]:
print train_data_features.shape

(25000, 5000)


In [18]:
vocab = vectorizer.get_feature_names()
print vocab[:10]

[u'abandoned', u'abc', u'abilities', u'ability', u'able', u'abraham', u'absence', u'absent', u'absolute', u'absolutely']


In [19]:
print "Training with Random Forest..."
from sklearn.ensemble import RandomForestClassifier

Training with Random Forest...


In [20]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(train_data_features,train["sentiment"])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
print 'Producing the cross validation score for the data..'
from sklearn.cross_validation import cross_val_score

Producing the cross validation score for the data..


In [22]:
score = cross_val_score(rfc,train_data_features,train["sentiment"],cv=5)

In [23]:
print '5 fold cv score : {}'.format(round(np.mean(score),3))

5 fold cv score : 0.843
